## Load Date

In [8]:
import pandas as pd
import numpy as np

data = pd.read_csv('/home/cloudcraftz/HandsOnPytorch/Data/data_banknote_authentication.txt', header=None)
data.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [61]:
X = data[[0,1,2,3]].values
Y = data[4].values

## Dataset & Dataloader

In [63]:
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self,X,y):
        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype = torch.float32)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]

        return x, y
    
    def __len__(self):
        return self.labels.shape[0]

In [64]:
train_size = int(X.shape[0]*0.80)
train_size

1097

In [65]:
val_size = X.shape[0] - train_size
val_size

275

In [66]:
dataset = MyDataset(X,Y)

torch.manual_seed(1)
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=10,
    shuffle=True,
)

val_loader = DataLoader(
    dataset=val_set,
    batch_size=10,
    shuffle=False,
)

In [67]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = torch.nn.Linear(in_features=num_features, out_features=1)

    def forward(self,x):
        logits = self.linear(x)
        probas = torch.sigmoid(logits)
        return probas


In [70]:
import torch.nn.functional as F

torch.manual_seed(1)

model = LogisticRegression(num_features=4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

num_epochs = 200

for epoch in range(num_epochs):
    
    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        probas = model(features)
       
        loss = F.binary_cross_entropy(probas, class_labels.view(probas.shape))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 20: # log every 20th batch
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
                   f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
                   f' | Loss: {loss:.2f}')

Epoch: 001/200 | Batch 000/110 | Loss: 1.30
Epoch: 001/200 | Batch 020/110 | Loss: 0.85
Epoch: 001/200 | Batch 040/110 | Loss: 0.66
Epoch: 001/200 | Batch 060/110 | Loss: 0.37
Epoch: 001/200 | Batch 080/110 | Loss: 0.28
Epoch: 001/200 | Batch 100/110 | Loss: 0.29
Epoch: 002/200 | Batch 000/110 | Loss: 0.39
Epoch: 002/200 | Batch 020/110 | Loss: 0.22
Epoch: 002/200 | Batch 040/110 | Loss: 0.35
Epoch: 002/200 | Batch 060/110 | Loss: 0.19
Epoch: 002/200 | Batch 080/110 | Loss: 0.19
Epoch: 002/200 | Batch 100/110 | Loss: 0.18
Epoch: 003/200 | Batch 000/110 | Loss: 0.13
Epoch: 003/200 | Batch 020/110 | Loss: 0.28
Epoch: 003/200 | Batch 040/110 | Loss: 0.21
Epoch: 003/200 | Batch 060/110 | Loss: 0.37
Epoch: 003/200 | Batch 080/110 | Loss: 0.18
Epoch: 003/200 | Batch 100/110 | Loss: 0.14
Epoch: 004/200 | Batch 000/110 | Loss: 0.15
Epoch: 004/200 | Batch 020/110 | Loss: 0.07
Epoch: 004/200 | Batch 040/110 | Loss: 0.17
Epoch: 004/200 | Batch 060/110 | Loss: 0.16
Epoch: 004/200 | Batch 080/110 |

In [71]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    
    correct = 0.0
    total_examples = 0
    
    for idx, (features, class_labels) in enumerate(dataloader):
        
        with torch.no_grad():
            probas = model(features)
        
        pred = torch.where(probas > 0.5, 1, 0)
        lab = class_labels.view(pred.shape).to(pred.dtype)

        compare = lab == pred
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

In [72]:
train_acc = compute_accuracy(model, train_loader)
print(f"Accuracy: {train_acc*100:.2f}%")

Accuracy: 98.72%


In [73]:
val_acc = compute_accuracy(model, val_loader)
print(f"Accuracy: {val_acc*100:.2f}%")

Accuracy: 99.27%
